## Install Faster-whisper and PytubeFix

In [1]:
!pip install faster-whisper -q
!pip install pytubefix -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 45.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 79.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.2/760.2 kB 13.3 MB/s eta 0:00:00a 0:00:01


## Download the Audio from Youtube

In [2]:
from pytubefix import YouTube
from pytubefix.cli import on_progress

In [3]:
def audio_download(urls):
    download_path = "/kaggle/working/"
    for url in urls:
        yt = YouTube(url, on_progress_callback=on_progress)
        ys = yt.streams.get_audio_only()
        ys.download(output_path=download_path)

In [4]:
songs_list = []

while True:
    url = input("Enter a YouTube URL (or 'done' to finish) :")
    if url == 'done':
        break
    songs_list.append(url)

Enter a YouTube URL (or 'done' to finish) : https://www.youtube.com/watch?v=2zlZ6bpyoM8
Enter a YouTube URL (or 'done' to finish) : done


In [5]:
audio_download(songs_list)

## Transcribe the list of Audios

In [6]:
import os
from faster_whisper import WhisperModel

In [7]:
model_size = "large-v3"

model = WhisperModel(model_size, device="cuda", compute_type="float16") # Use Settings -> Accelerator -> GPU T4 x2

vocabulary.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

In [8]:
folder_path = "/kaggle/working/"
folder_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.mp3', '.m4a', '.wav'))]

In [9]:
for song in folder_list:
    segments, info = model.transcribe(song, beam_size=5)
    song_basename = os.path.basename(song)

    print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

    for segment in segments:
        with open(song_basename + ".txt", "a") as file:
            file.write("[%.2fs -> %.2fs] %s\n" % (segment.start, segment.end, segment.text))

Detected language 'en' with probability 0.963867
